In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import confusion_matrix, classification_report,f1_score
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
import numpy as np
from scipy.spatial.distance import cdist
import heapq
from collections import Counter
import os
import ast
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import random
import lime
random.seed(1993)
random.seed(1993)
%matplotlib inline

In [2]:
df_test = pd.read_csv("../data/span/test/test.tsv",sep="\t")
df_test

,text,spans
0,Evanukachum pondati kolandhainu sentiment irun...,"[71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 8..."
1,Adei ennada short film la irunthu suturukinga,"[34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]"
2,Super dialogue Oruthar mela visvasam kattrath...,"[68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 7..."
3,Epadiya jathi padam. Ponnu padama edungada. In...,"[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 7..."
4,Ponnu mella kaivaikuravan Kai mattum illa uyir...,"[42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 5..."
...,...,...
871,காலா.காபலி.அசுரன்.பாரியேரும்.பெருமள் வந்த அப்ப...,"[101, 102, 103, 104, 105, 106, 107, 108, 109, ..."
872,Kekka bekka short film mathri irukey,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]"
873,Bayangaram... Trailerey ippadina appa Padam en...,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
874,komali rasini vesam pottu tamil ilichavayangal...,"[0, 1, 2, 3, 4, 5, 6, 7, 31, 32, 33, 34, 35, 3..."


In [3]:
# Complex + Multilabel

In [4]:
# Read the output and create spans

In [5]:
def f1_span(predictions, gold):
    """
    F1 (a.k.a. DICE) operating on two lists of offsets (e.g., character).
    >>> assert f1([0, 1, 4, 5], [0, 1, 6]) == 0.5714285714285714
    :param predictions: a list of predicted offsets
    :param gold: a list of offsets serving as the ground truth
    :return: a score between 0 and 1
    """
    if len(gold) == 0:
        if len(predictions) == 0:
            return 0
        else:
            return 1
    if len(predictions) == 0:
        return 0.
    predictions_set = set(predictions)
    gold_set = set(gold)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)

In [6]:
# Complex-Multi

In [94]:
import re


def eval_multiple(LIMEFILE,MAXSIZE,MINSIZE):

    def get_spans_lime(word_list,index):
        THRESH=-0.3
        df_test = pd.read_csv("../data/span/test/test.tsv",sep="\t")
        ref_text = df_test["text"].values.tolist()[index]
        spans =  [[ref_text.find(words),ref_text.find(words)+len(words)] for words,score in word_list if score>THRESH]
        complete_spans=[]
        for spn in spans:
            complete_spans+=range(spn[0],spn[1]+1)
        return (df_test["spans"][index], sorted(complete_spans))

    output_xlm_cm = np.load(LIMEFILE,allow_pickle=True)
    output_xlm_cm = output_xlm_cm.tolist() 


    total_sum=[]
    for index, lime_pred in enumerate(output_xlm_cm):
        if len(df_test["text"].values.tolist()[index])>=MINSIZE and len(df_test["text"].values.tolist()[index])<=MAXSIZE:    
            print(lime_pred)
            gt,pred = get_spans_lime(lime_pred[0],index)
            gt=ast.literal_eval(gt)
            total_sum.append(f1_span(pred,gt))
    print("File:",LIMEFILE, "Results:", np.average(total_sum))

LIMEFILE = "../models/complex_cue_multi/offensive_bert_complex_aug_multi/bert_lime_complex_multi_all.npy"

MAXSIZE = 10000
MINSIZE = 0
eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

MAXSIZE = 30
MINSIZE = 0
eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

MAXSIZE = 50
MINSIZE = 30
eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

MAXSIZE = 500
MINSIZE = 50
eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

[[('mata', 0.002831786324617167), ('kollama', -0.0023053598398121685), ('vida', 0.0020121624617167556), ('Evanukachum', 0.0013062340706348921), ('sentiment', 0.0009245109364676738), ('kola', 0.000503027600929375)]]
[[('ennada', 0.048646841259068346), ('Adei', -0.04196413850588112), ('suturukinga', 0.020990792660700752), ('short', -0.012266300253580126), ('la', 0.006712073113900097), ('film', -0.003827844734960866)]]
[[('paduthathinga', 0.0742039527255436), ('asinga', -0.022182149875230872), ('Oruthar', -0.01347400823490209), ('mela', -0.012445448677072991), ('dialogue', -0.010538043536989755), ('Super', -0.010412262581126333)]]
[[('parambarai', -0.0017790331393706033), ('Epadiya', 0.0009141360085685203), ('edungada', -0.0008122798486602772), ('Anda', -0.0007459098558521101), ('jathi', 0.0005461131378872337), ('nu', 0.00047015432918391366)]]
[[('mella', 0.004184437740046943), ('kaivaikuravan', 0.0034754401411763473), ('Kai', 0.002595139594258174), ('illa', 0.0015529054133906822), ('nanb

[[('madhiri', -0.0027466467361677667), ('therila', 0.0014386448847158318), ('raghavan', -0.0009379637560169014), ('selva', 0.0007883666861396188), ('film', -0.0005879533230245207), ('Idhu', 2.6998836647567037e-05)]]
[[('panni', 0.10216120066633089), ('Nallave', 0.08467162105040209), ('keela', -0.0414749140750467), ('ippo', -0.03119158230231736), ('vijay', -0.026912623078244848), ('sethu', -0.02433695299304996)]]
[[('Yeppa', 0.03974825580121026), ('kattunge', -0.035039317834403394), ('rajni', 0.03228024450000497), ('pavam', 0.030211072878036368), ('Konjam', -0.02985030382673605), ('karuna', -0.023213229663831176)]]
[[('orray', 0.0032170967541187935), ('eppavumay', 0.001867801678596265), ('mathiri', 0.0016227087207187092), ('Kavalamana', -0.0016132797000993106), ('intro', -0.0008412380930238585), ('irru', 0.0006700112083121642)]]
[[('bad', 0.0050371266487689624), ('dad', -0.00400826076763762), ('ur', 0.002841721627285521), ('am', -0.002053365083911357), ('are', -0.0020404766331040053), (

[[('amma', 0.0034130688678382886), ('dhadava', -0.0007762902388633923), ('da', 0.0007657564981739632), ('ethana', -0.0006789619687560832), ('vah', 0.0003346544897238234), ('saagadipinga', 1.602865780725119e-05)]]
[[('irukku', 0.02614317104508876), ('kevalama', -0.015195991337163416), ('pannathinga', 0.006209060426549535), ('Thayavusenchi', 0.004927298625665728), ('da', 0.004029026181238983), ('ya', 0.0037165097482025544)]]
[[('pannotiyeda', 0.04562934152470197), ('Dai', -0.021881712090938032), ('Super', -0.02158956837465724), ('ah', 0.021402479586174143), ('pangam', 0.019482288319431898), ('director', 0.0139835386101953)]]
[[('naksal', -0.008198470992622105), ('Urimaiyai', 0.005435379258693265), ('ketpavan', 0.005123783402596293), ('sonnal', 0.0031806624096160164), ('antha', 0.0015897334494168812), ('urimaiyai', 0.001524343508763567)]]
[[('dance', 0.004270856824878117), ('niruthika', -0.0015626783127932527), ('varalala', -0.0011959308317618666), ('unaku', -0.0010734019372748084), ('tha

[[('Nee', 0.012455631977089104), ('captain', -0.01038454705288272), ('America', -0.008827193768096103), ('enna', -0.006793556707490209), ('va', -0.006427521398745601)]]
[[('movie', -0.002825277818253428), ('relies', 0.0025905434034756455), ('make', -0.00196517169918561), ('dont', 0.0019612512727182323), ('Why', -0.0015755245489671801), ('you', 0.0011598422711189623)]]
[[('y', 0.048898297326500056), ('blood', -0.032877192728237296), ('la', -0.025306929123931732), ('aanadhi', 0.019632640791873306), ('same', -0.017904851481148204), ('modulation', 0.01729966207510945)]]
[[('odachudalam', -0.001346038016034771), ('absolutely', 0.0013131198634759925), ('mudiyadhu', -0.0009178826247692121), ('azhikka', 0.0008119185775015482), ('azhichudalam', 0.0006963825974506876), ('avanukku', 0.0006920624307199155)]]
[[('Dislike', 0.0028005580824872296), ('pottavangalam', 0.0015140713727370134), ('Unga', 0.0006506293880020191), ('trailer', 0.0005383959822987435), ('vachi', 0.00042511223420134354), ('nengil

[[('thambi', -0.0014932121619922235), ('built', 0.0013427859253195029), ('up', -0.000739479068785588), ('Dei', -0.0007354746673216429), ('ah', 0.0006086289425265688), ('padamachu', -0.0005727218407059482)]]
[[('nadichutu', 0.02257324387324563), ('la', 0.02004630631319334), ('adha', -0.018726098424451994), ('mokka', 0.018580112730423953), ('shivakuda', -0.016328114898331744), ('movies', 0.01624106322404581)]]
[[('kudhathu', -0.0076817981852584), ('panne', 0.007673073457289249), ('vechiruke', -0.004849436064910375), ('skip', -0.004434357667748239), ('eh', -0.004061315451830784), ('trailer', -0.0031830936217787037)]]
[[('likes', -0.0026613012753601423), ('என', 0.0010342808257337661), ('44K', 0.0008924017020826103), ('But', -0.0007375075776453611), ('55K', 0.0006723416197831986), ('views', -0.00038338599330959026)]]
[[('dae', 0.0036782490084721234), ('nalla', 0.0013956975910644149), ('unmaya', 0.0011448577859788226), ('illa', 0.0010497654291006554), ('sathyama', -0.0010201165109882194), ('

...

[[('daughter', 0.00260947582585265), ('edho', 0.002153227647563682), ('Akshara', -0.0015739230445563608), ('sir', -0.0014521539370471615), ('irundhu', 0.0012309655862614804), ('vikram', -0.0009789605385845006)]]
[[('marana', 0.007226933556337897), ('adichi', -0.006667544345754069), ('oda', -0.005725983017206377), ('style', 0.005060363367713094), ('underwear', -0.004934523453419431), ('semma', 0.004252169622762066)]]
[[('thuu', -0.06670253398355867), ('mama', 0.05487504333916376), ('vaaii', 0.051982154960397216), ('pu', -0.021820140734974895), ('kudathu', 0.02131646290254384), ('pesave', 0.013882755531367592)]]
[[('Indha', -0.003660417418504435), ('ungalukku', 0.002312372790090239), ('vera', -0.0013758533046051285), ('saniyana', -0.0013716011966576386), ('kidaikkaliya', 0.0008872838030143804), ('thavira', -0.0006721140117181724)]]
[[('eppo', 0.010882645064993035), ('2016', 0.009662245352484257), ('Rajni', -0.008307389543496909), ('la', 0.007128895846598404), ('announce', -0.005591526273

[[('anils', -0.0019747861614741297), ('irukunga', -0.0012068681007445503), ('school', 0.0011660032596015842), ('vandhutu', -0.0005934925661474387), ('dislikes', -0.00046907179278837314), ('irundhu', -0.0002401289451073403)]]
[[('Sasta', 0.01036582804355949), ('make', 0.006969852732968938), ('looks', 0.0063868011407701), ('up', 0.006044517462684192), ('cartoon', -0.004148526409518199), ('like', -0.0031527923152319505)]]
[[('Produce', 0.004139247145269266), ('Tamilar', 0.0033306684300401404), ('tamilar', 0.0031100111788121667), ('kedaichara', 0.0018800163247810289), ('reddy', -0.0013042289611306383), ('nu', -0.0010392602836774196)]]
[[('Chee', -0.07279387545222321), ('thoo', -0.023156279419302234), ('kooda', 0.012748692442318067), ('ivlo', 0.011978866806244427), ('Like', -0.011647087586646976), ('like', -0.008665713528223321)]]
[[('Full', 0.004363946855298116), ('story', 0.002896004262729685), ('ye', 0.0019810571553256265), ('sollitinga', -0.0018718729198630604), ('varuvan', -0.001651188

[[('Jayam', 0.004979896963404557), ('Ravi', 0.004902041926768185), ('Hollywood', 0.0037294776770658753), ('Verum', 0.002767426525725033), ('padathoda', -0.0016383444452705187), ('endha', -0.0012579203415816749)]]
[[('movie', 0.0012868291300378476), ('Pariyerum', -0.001010240390228811), ('varala', -0.0008548520574646317), ('award', 0.00043463282886257114), ('Perumal', -0.0004236550732809285), ('Ku', -0.0004187947795118779)]]
[[('ms', 0.0015837465077267856), ('padam', -0.001275792098082587), ('baskar', -0.0011135244802870062), ('Type', 0.0010433026229868727), ('Cringe', 0.0006363385107303433), ('cringe', 0.0005767690938861898)]]
[[('Ommaley', -0.06281477124754885), ('manusan', 0.024478990514487417), ('intha', 0.018202726210441427), ('ya', 0.016958253942693263), ('paraak', -0.013610355591024416), ('vintage', 0.009183917560921325)]]
[[('podunga', -0.004370376696629934), ('vayila', 0.003414333156147785), ('vecchu', 0.0032990273623117084), ('like', -0.002914613568019199), ('sunni', 0.0016266

[[('Agith', -0.009519593406621974), ('poi', -0.009400965421278534), ('oru', -0.009071820011830033), ('Vera', 0.008590629837191726), ('avanuku', -0.008384500737567047), ('venuma', 0.008230263152108471)]]
[[('munnadi', 0.002112291603121324), ('two', 0.001132494610440275), ('likes', -0.0010768415068748436), ('puriyala', -0.00104236504897399), ('vanthuruchu', -0.0006900740501548299), ('500k', -0.0006871677194763943)]]
[[('trailer', 0.051765750559603786), ('a', -0.02111894943460493), ('irukkum', 0.019873417182553456), ('pola', 0.015085444150441966), ('kevalama', -0.009962383423460284), ('vida', -0.008467120248481834)]]
[[('thiii', -0.001241655802731371), ('copy', -0.0011512937634622926), ('yosinga', -0.0006347270127520288), ('poduringa', -0.0005966904146645648), ('ungaluku', -0.0005719107145720768), ('Ajith', -0.0004993471391408438)]]
[[('Yenna', 0.006903163209819466), ('aal', 0.006588059330902731), ('da', -0.0045458862042005565), ('nachchunu', 0.0030944720190379978), ('Das', 0.001693798167

[[('thevidiya', -0.003950338373699018), ('payan', 0.0007674017432077108), ('baadu', 0.0005780288405984062), ('myskin', 0.000451969182198277), ('eduthurukan', 0.0003946260382852223), ('apadiye', -0.0002399485560556308)]]
[[('music', 0.0019570830050054475), ('yaarellam', 0.0008067278962229687), ('panringa', -0.0008016252585456198), ('feel', 0.0008008504264888881), ('maari', 0.0004915081421560815), ('Kaththi', 0.0003235371536552402)]]
[[('caste', 0.0018268008289359912), ('echakala', -0.0012531989353557632), ('thaguthiku', -0.0010213820380977666), ('sandaya', 0.0006000956301198311), ('all', 0.0005948770144633702), ('Ungaluku', 0.0005613091018596256)]]
[[('Andha', 0.047750447010888306), ('loose', -0.020159161426668702), ('Abhirami', 0.010067236426438732), ('Like', -0.007440668598852301), ('irruka', 0.006444974651994694), ('podunga', -0.0017764649285695673)]]
[[('Editing', 0.05407246971261079), ('kevalam', -0.03200493430246213), ('anna', -0.009494976544880322), ('kammiya', 0.0088814255443639

In [86]:
# Baseline and Complex

In [26]:
import re

def eval_multiple(LIMEFILE,MAXSIZE,MINSIZE):

    def get_spans_lime(word_list,index):
        word_list = [word_list]
        THRESH=-200
        df_test = pd.read_csv("../data/span/test/test.tsv",sep="\t")
        ref_text = df_test["text"].values.tolist()[index]
        spans =  [[ref_text.find(words),ref_text.find(words)+len(words)] for words,score in word_list if score>THRESH]
        complete_spans=[]
        for spn in spans:
            complete_spans+=range(spn[0],spn[1]+1)
        return (df_test["spans"][index], sorted(complete_spans))

    output_xlm_cm = np.load(LIMEFILE,allow_pickle=True)
    output_xlm_cm = output_xlm_cm.tolist() 


    total_sum=[]
    for index, lime_pred in enumerate(output_xlm_cm):
        if len(df_test["text"].values.tolist()[index])>=MINSIZE and len(df_test["text"].values.tolist()[index])<=MAXSIZE:    
            gt,pred = get_spans_lime(lime_pred[0],index)
            gt=ast.literal_eval(gt)
            aa = f1_span(pred,gt)
            if aa>0.1:
                print(df_test["text"].values.tolist()[index],lime_pred)
            total_sum.append(aa)
    print("File:",LIMEFILE, "Results:", np.average(total_sum))

LIMEFILE = "../models/baseline/offensive_xlm_baseline/xlm_lime_baseline.npy"

# MAXSIZE = 10000
# MINSIZE = 0
# eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

MAXSIZE = 500
MINSIZE = 50
eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

# MAXSIZE = 50
# MINSIZE = 30
# eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

# MAXSIZE = 500
# MINSIZE = 50
# eval_multiple(LIMEFILE,MAXSIZE,MINSIZE)

Naan arasilaluku varuvadhu urudhi nu rajini solrathu 2017 Dec, aana story line 2016 nu solreenga. Enna da trailer la ye flaw irruku [('flaw', 0.06473631106667564), ('da', 0.036867528061762196), ('Enna', 0.02275438387467514), ('solreenga', 0.017475237383559366), ('rajini', 0.012137752840242107), ('aana', 0.011896939621566566)]
Epidi daa veakame illama unlike poduringa...  Padam blockbuster daa [('unlike', 0.25951108007074364), ('blockbuster', -0.09093038918511587), ('Epidi', 0.03205029804896043), ('veakame', 0.02456590270879732), ('illama', 0.02362559869597955), ('daa', 0.018142885264184623)]
Muttal makkal,makkaluku Evan enne seithan.sanbathithu avane vachukuvan. Gaja puyaluku ena seithan. Makkal than mural. [('vachukuvan', 0.06912466903371), ('Makkal', -0.06599573085605023), ('avane', 0.058315073847574214), ('mural', 0.052700647702278135), ('than', 0.045968965972825376), ('Muttal', 0.038088316166709935)]
Dei Yappa munjha paarthu Sirichu Sirichu vayiru vazhikuthu 🤣 [('Dei', 0.1610403331

Beethoven song uh matu Illayaraja use panraru...Idhu Enna polapu [('Enna', 0.06843206823058832), ('Beethoven', 0.06668019694104327), ('use', 0.0540610199936459), ('panraru', 0.04388899786635449), ('Idhu', 0.029865260415164872), ('matu', 0.023209954043640808)]
enna katha endum puriyalla. mokka padamathaaan irukkummmmmm [('mokka', 0.3385076802635185), ('endum', 0.014936363954534355), ('katha', 0.013518354789507546), ('enna', 0.012246321663012528), ('puriyalla', -0.012174964961602877), ('padamathaaan', 0.01045405164816551)]
Entha padan veluyana aprom athigamabnadajum.nqngal ayudam eduka thayaar [('eduka', 0.10753248898011294), ('thayaar', 0.06914769666929575), ('Entha', 0.058932120822510335), ('athigamabnadajum', 0.05314813621390381), ('veluyana', 0.032071820757222445), ('nqngal', -0.02463471915961152)]
Ini kunjith ku aappuu dhaan......avan pozhapula kai vechaachu..... [('kunjith', 0.08464652888077204), ('vechaachu', 0.0776776596293015), ('kai', 0.046226225317042124), ('Ini', 0.0418519114

Semma super trailer. Panakara veetu pullaiya kadhal ngra perula Kalyanam panikitu life la settle aagara naadharigaluku semma serupadi movie. [('semma', -0.07862003788476751), ('pullaiya', 0.06474033677896575), ('serupadi', 0.06079507699902757), ('panikitu', 0.060385743136089506), ('super', -0.045620495122925646), ('veetu', 0.036585628210727236)]
Padam bayankara kodurama irunthalum.. Udhaya nidhi patha sirupu tha varuthu.. [('kodurama', 0.06483181190846174), ('irunthalum', 0.026786141220434937), ('nidhi', 0.02415358829787478), ('Udhaya', 0.02000081564703311), ('bayankara', 0.019697780084684428), ('Padam', 0.019392913890036844)]
Vivek Oberoi's Dubbing Voice kevalaaama Irukku... Adhukku Avanaay Nalla paessuvaaan... [('kevalaaama', 0.07959160676150341), ('Nalla', -0.06530192444025763), ('Voice', 0.036864889188316353), ('Irukku', 0.032790263383068466), ('Adhukku', 0.03158406585659278), ('Dubbing', 0.027533164315683395)]
Nadakkadhu, nadakkakoodathu, nadakka vidamattaen! Who expected that lin

ada pundegale unnu mada intha peee pundai padathle yadkerenga. ada tuuuu [('pundai', 0.13823439263530454), ('pundegale', 0.10495168789584629), ('yadkerenga', 0.02492759939064699), ('ada', 0.01802336204150244), ('unnu', 0.016772423343201812), ('intha', 0.013003863966450068)]
Nan arasiyaluku varrathu uruthinu 1996 la Rajini sonna proof ethachum iruntha Jayam Ravi release panatum. Ilana Jayam Ravi oru fake thaayoli mavan than apdingurathu uruthi aayidum.   Ithula 2017 kum 2016 kum kooda vithyasam theriyatha thaththi punthaingala irunganunga padam eduthavanunga karumam [('fake', 0.09092046034290965), ('punthaingala', 0.012140343609630951), ('Nan', -0.011720066036301385), ('theriyatha', -0.010570114922495356), ('1996', 0.008704474036871147), ('mavan', 0.007881998596213574)]
Thuppakki padathaium theri padaththaium peranmai padaththaium copy adichi irukkanuga puthusa yedukkave maattingala [('copy', 0.17028156059903024), ('adichi', 0.07662220792611359), ('maattingala', 0.0470573848246441), ('i

Adei... youtube ethuku Hang ana 483k laya 30 min ah iruku [('Adei', 0.10402966920089732), ('ethuku', 0.06749655674258728), ('min', -0.058960597260565406), ('iruku', 0.05698981529373708), ('483k', 0.052296406759226345), ('ah', 0.04784028745983586)]
Trailer lam nalla irukku...!! Aana padatha release panna maatringale pa..... [('maatringale', 0.12715610732348612), ('irukku', 0.053292548856679625), ('panna', 0.04241889999807569), ('Aana', 0.04197267394735733), ('Trailer', 0.028233284227752235), ('lam', 0.025631843812524365)]
anils dislikes  school la irundhu vandhutu irukunga [('dislikes', 0.149435254273251), ('anils', 0.10054364004166798), ('school', -0.01813832708313991), ('irundhu', 0.014970473283529028), ('vandhutu', 0.013914986069417675), ('la', 0.009099976824803133)]
Sasta make up... Older version of jailalitha looks like cartoon [('cartoon', 0.1907935276301249), ('looks', 0.09217331933578662), ('up', 0.04264680175351971), ('make', 0.03559534495942208), ('Older', 0.024950780686550586

panni koottam maathiri screen fulla aatkala irukkanga , trailer average. [('panni', 0.05959785269384556), ('maathiri', 0.05203732788194792), ('average', 0.046518917399720854), ('irukkanga', 0.03505842970454353), ('screen', 0.02737306684431468), ('fulla', 0.01860705835867519)]
Massukku porundhura patthu poruthamum unkittadha iruku. ungakitta mattomdha iruku thalaivan DA... [('thalaivan', -0.10046368461289397), ('ungakitta', 0.06315222740627077), ('porundhura', 0.04355909144508828), ('DA', 0.0427078098500772), ('Massukku', -0.038079427839722006), ('unkittadha', 0.03462235158191414)]
Entha devangu naaiuva ennum uiroda than erukkudhugala [('devangu', 0.07878717870143043), ('erukkudhugala', 0.06005136603917101), ('uiroda', 0.048240455632666705), ('Entha', 0.028488668076377448), ('naaiuva', 0.018104313462196358), ('than', 0.003611548510956362)]
Peeta parakkk thalivarr marana massss trailer  Kolle ghandle irrke RAJINI ... DAAAA..... [('irrke', 0.056671308466738395), ('Kolle', 0.05591412953260

Trailer'eh oru mairu logic illama irukke, padam yenna kevala pundaya irukka potho 🤣🤣 [('pundaya', 0.19621006158248905), ('kevala', 0.024887840621901677), ('potho', 0.01900139936422216), ('mairu', 0.013791806953082507), ('yenna', 0.013139809954808353), ('eh', 0.012108247942676061)]
Mahanadi mari try pannanum nu eppadi Barbie doll luku pacha saree potta mari pannivachurukinga [('pannivachurukinga', 0.08036224430654383), ('Barbie', 0.07439322038339641), ('potta', 0.04092711104964905), ('eppadi', 0.03957187678197628), ('pacha', 0.03408338450822835), ('Mahanadi', 0.024970652131802364)]
Yenakku mattum than iru mugan movie  Maari thonudha? [('thonudha', 0.07991351916102833), ('mattum', -0.023040840181161055), ('mugan', 0.02217972689836408), ('Maari', 0.017597155047474226), ('Yenakku', 0.01699273516882051), ('iru', 0.012229194452222453)]
dai potta others fans mooditu pongada  singam da VANNIYAN DA [('dai', 0.11223014421006294), ('mooditu', 0.07158439044940675), ('pongada', 0.0537856149603074),